In [56]:
import json
import os

import polars as pl

In [57]:
data_path = "data"
tabular_data_path = os.path.join(data_path, "tabular")
vocabulary_path = os.path.join(data_path, "vocabulary.json")

In [58]:
splits = ["train", "validation", "test"]
with open(vocabulary_path) as f:
    vocabulary = json.load(f)

In [59]:
multi_hot = [pl.col("description").str.split(" ").list.contains(word).cast(pl.Int8).alias(word) for word in vocabulary]

In [60]:
for split in splits:
    dataset_path = os.path.join(tabular_data_path, f"{split}.csv")
    processed_data_path = os.path.join(tabular_data_path, f"{split}_processed.csv")
    dataset = pl.read_csv(dataset_path)
    if "description" in dataset.columns:
        dataset = dataset.with_columns(multi_hot).drop("description").select(pl.exclude("class"), pl.col("class"))
    dataset.write_csv(processed_data_path)